In [39]:
import os
import re
import subprocess

from datetime import timedelta

import googleapiclient.discovery
import feedparser

In [30]:
TIME_30_MINS = 30*60
TIME_4_HRS = 4*60*60

def sanity_check_duration(duration, title=None):
    if duration < TIME_30_MINS:
        print(f'{duration} suspiciously short for {title}')
    if TIME_4_HRS < duration:
        print(f'{duration} suspiciously long for {title}')

----
# YouTube

https://developers.google.com/youtube/v3/docs

https://developers.google.com/youtube/v3/quickstart/python

In [2]:
# playlist constants

SRC_YT_NGWD = "https://www.youtube.com/playlist?list=PLz3Be--ot61PFrRm767OdkgnHPBKg4gy4"
SRC_YT_PF_PLAYTEST = "https://www.youtube.com/playlist?list=PLz3Be--ot61NBeeto8pmgQDBad34I1tzb"
SRC_YT_STRANGE_AEONS = "https://www.youtube.com/playlist?list=PLz3Be--ot61NuA-PaoFDt-GlKOSbxQIfq"
SRC_YT_THUNDER_COMP = "https://www.youtube.com/playlist?list=PLz3Be--ot61N8ew8AI41YZPPgUtIJr_D0"

In [3]:
YT_API_KEY = os.environ.get("YT_API_KEY", "UNDEFINED")
YT_CHANNEL_ID = "UC83CJFLyDe72XgkKBd5a9IA"

yt_all_playlist_links = f'''
{SRC_YT_NGWD}
{SRC_YT_PF_PLAYTEST}
{SRC_YT_STRANGE_AEONS}
{SRC_YT_THUNDER_COMP}
'''

In [4]:
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = YT_API_KEY
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey = DEVELOPER_KEY
)

In [5]:
request = youtube.playlists().list(
    part="id,contentDetails,snippet",
    channelId=YT_CHANNEL_ID,
    maxResults=50,
)
playlists_response = request.execute()
playlists_response

{'kind': 'youtube#playlistListResponse',
 'etag': 'LOkyDmyXSpbyn4qsVbR1UfvgpUc',
 'pageInfo': {'totalResults': 34, 'resultsPerPage': 50},
 'items': [{'kind': 'youtube#playlist',
   'etag': 'c0EdV2heZdaNfCrZCgnbJ9CXkcE',
   'id': 'PLz3Be--ot61PFrRm767OdkgnHPBKg4gy4',
   'snippet': {'publishedAt': '2021-01-11T13:25:09Z',
    'channelId': 'UC83CJFLyDe72XgkKBd5a9IA',
    'title': 'New Game, Who Dis?',
    'description': '',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/ks3meUecUyg/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/ks3meUecUyg/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/ks3meUecUyg/hqdefault.jpg',
      'width': 480,
      'height': 360},
     'standard': {'url': 'https://i.ytimg.com/vi/ks3meUecUyg/sddefault.jpg',
      'width': 640,
      'height': 480},
     'maxres': {'url': 'https://i.ytimg.com/vi/ks3meUecUyg/maxresdefault.jpg',
      'width': 12

In [6]:
all_durations = []

for playlist in playlists_response['items']:
    playlist_id =  playlist['id']
    if playlist_id not in yt_all_playlist_links:
        print(f'skipped playlist {playlist_id}')
        continue
    else:
        print(f'examining playlist {playlist_id}')
    
    request = youtube.playlistItems().list(
        part="id,contentDetails,snippet",
        playlistId=playlist_id,
        maxResults=50,
    )
    playlist_response = request.execute()
    
    for video in playlist_response['items']:
        video_id = video['snippet']['resourceId']['videoId']
        print(f'checking video {video_id}')
        request = youtube.videos().list(
            part="contentDetails",
            id=video_id
        )
        video_response = request.execute()
        
        duration = [item['contentDetails']['duration'] for item in video_response['items']]
        all_durations.extend(duration)
    
    print()

all_durations

examining playlist PLz3Be--ot61PFrRm767OdkgnHPBKg4gy4
checking video ks3meUecUyg
checking video NZRR4ZYyemo
checking video wMGXfTo2ZCY
checking video a2J4Yd9KQEs
checking video Rjo0Fyk7Kng
checking video l_rILKBYChk
checking video v9qOIRt4CxU
checking video w7Bc0R7MYpA
checking video azy5Rx3l2uc
checking video uEpQXjCD42o
checking video 3MmKryyM63Q
checking video rVnfByev1ak
checking video Of6gvux_HYQ
checking video HsWls7iSCgw
checking video JOCn67WIkHU
checking video 6CGClvvfgNA
checking video xjeigXyvTPA
checking video -r83EOM68uM
checking video 6nhptYbgqS4
checking video _-etvSHVfMg
checking video xapiGy39Oms
checking video IVlLmyR8U90
checking video OOvpa8Go5Nk
checking video m-sLFZrGZgU
checking video hl_bRCIvMfk
checking video zc5s-z2owGk
checking video thxtMQPc4Kc
checking video iN9TDtBFwCc
checking video E6iumoIi2RM
checking video ewhbcqDyw00
checking video 6fC_HYu7Qis

examining playlist PLz3Be--ot61N8ew8AI41YZPPgUtIJr_D0
checking video Z63SMouuiJ0
checking video ZPQELTWLvSs


['PT2H34M4S',
 'PT2H6M45S',
 'PT2H59M16S',
 'PT2H32M35S',
 'PT2H12M18S',
 'PT2H24M11S',
 'PT2H23M53S',
 'PT2H18M9S',
 'PT2H10M30S',
 'PT2H36M46S',
 'PT2H27M15S',
 'PT3H11M5S',
 'PT2H44M36S',
 'PT1H46M23S',
 'PT2H39M41S',
 'PT2H16M2S',
 'PT2H15M50S',
 'PT2H15M7S',
 'PT2H22M52S',
 'PT1H54M8S',
 'PT1H42M42S',
 'PT2H42M43S',
 'PT2H22M15S',
 'PT3H1M5S',
 'PT2H37M43S',
 'PT2H3M55S',
 'PT2H7M15S',
 'PT2H27M55S',
 'PT2H19M48S',
 'PT2H38M54S',
 'PT2H11M41S',
 'PT2H44M17S',
 'PT2H40M26S',
 'PT2H37M28S',
 'PT3H38M7S',
 'PT3H6M4S',
 'PT2H2M41S',
 'PT55M5S',
 'PT2H7M38S',
 'PT1H56M22S',
 'PT1H58M18S',
 'PT2H22M19S',
 'PT2H13M29S',
 'PT2H6M28S',
 'PT2H4M16S',
 'PT2H1M35S',
 'PT2H19M',
 'PT2H14M37S',
 'PT2H21M52S',
 'PT2H17M26S',
 'PT2H22M52S',
 'PT2H4M35S',
 'PT2H25M10S',
 'PT3H37M29S',
 'PT3H58M23S',
 'PT3H58M59S',
 'PT4H55M49S',
 'PT4H57M12S',
 'PT2H19M23S',
 'PT2H21M55S',
 'PT4H51M10S',
 'PT5H13M40S',
 'PT5H24M15S',
 'PT5H3M1S',
 'PT1H53M14S',
 'PT5H23M16S',
 'PT2H31M57S',
 'PT4H35M29S',
 'PT4H59

In [7]:
time_components = [re.findall('[^A-Z]+[A-Z]', duration) for duration in all_durations]

long_videos = [
    60*60*int(time[0][:-1]) + 60*int(time[1][:-1]) + int(time[2][:-1])
    for time in time_components if len(time) == 3
]
short_videos = [
    60*int(time[0][:-1]) + int(time[1][:-1])
    for time in time_components if len(time) == 2
]

[sanity_check_duration(duration) for duartion in long_videos]
[sanity_check_duration(duration) for duartion in short_videos]

full_video_time = sum(long_videos) + sum(short_videos)
full_video_time

680104

----
# RSS Feeds

https://feedparser.readthedocs.io/en/latest/introduction.html

In [8]:
SRC_RSS_GCP = "https://feeds.blubrry.com/feeds/the_glass_cannon.xml"
SRC_RSS_ANA = "http://feeds.megaphone.fm/androids-and-aliens"
SRC_RSS_SQSS = "https://anchor.fm/s/47417ac0/podcast/rss"
SRC_RSS_FOD = "https://feeds.blubrry.com/feeds/cannonfodder.xml"

In [33]:
def get_ffprobe_duration(file_path):
    return [
        'ffprobe',
        '-v', 'error',
        '-show_entries', 'format=duration',
        '-of', 'default=noprint_wrappers=1:nokey=1',
        file_path
    ]
def get_stdout_int(stdout):
    return int(float(stdout.decode('utf-8').strip()))

def get_duration_for_entry(entry):
    ffprobe_result = subprocess.run(
        get_ffprobe_duration(entry['links'][-1]['href']),
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT
    )
    entry_duration = get_stdout_int(ffprobe_result.stdout)
    sanity_check_duration(entry_duration, title=entry['title'])
    return entry_duration

In [34]:
d = feedparser.parse(SRC_RSS_GCP)

(d['feed']['title'],
 d['entries'][0]['title'],
 d['entries'][0]['itunes_duration'],
 d['entries'][0]['links'][-1]['length']
)

full_gcp_time = 0
print(f"{len(d['entries'])} episodes")

for entry in d['entries']:
    if 'itunes_duration' not in entry:
        print(f"missing itunes_duration: {entry['title']}")

        entry_duration = get_duration_for_entry(entry)    
        full_gcp_time += entry_duration
        print(f"ffprobe found duration {entry_duration}")
        
        continue
    
    time_components = entry['itunes_duration'].split(':')
    
    if len(time_components) == 3:
        h, m, s = time_components
    if len(time_components) == 2:
        m, s = time_components
        h = 0
    
    entry_duration = 60*60*int(h) + 60*int(m) + int(s)
    sanity_check_duration(entry_duration, title=entry['title'])
    if entry_duration < 100:
        print(f"correcting low duration {entry_duration} for ep {entry['title']}")
        entry_duration = get_duration_for_entry(entry)    
        full_gcp_time += entry_duration
        print(f"ffprobe found duration {entry_duration}")
        
    full_gcp_time += entry_duration

full_gcp_time

295 episodes
missing itunes_duration: Episode 168 - A Tale of Frost and Flame
ffprobe found duration 3352
missing itunes_duration: Episode 167 - Church and Destroy
ffprobe found duration 4601
missing itunes_duration: Episode 166 - Chapel of My Eye
ffprobe found duration 4307
missing itunes_duration: Episode 165 - Mummy Issues
ffprobe found duration 4365
missing itunes_duration: Episode 164 - The Temple of Detrimental Evil
ffprobe found duration 3752
missing itunes_duration: Episode 163 - Bear With Us
ffprobe found duration 4461
missing itunes_duration: Episode 162 - Half-Orc Will Travel 2: When Nature Kalls
ffprobe found duration 4992
missing itunes_duration: Episode 161 - Of Scythe and Men
ffprobe found duration 3572
missing itunes_duration: Episode 160 - Welcome Back, Slaughter
ffprobe found duration 4255
missing itunes_duration: Episode 159 - Blood From a Stone Shape
ffprobe found duration 3657
missing itunes_duration: Episode 158 - Bring in 'da Noise, Bring in 'da Monk
ffprobe foun

1336605

In [35]:
e = feedparser.parse(SRC_RSS_ANA)

(e['feed']['title'],
 e['entries'][0]['title'],
 e['entries'][0]['itunes_duration'],
 e['entries'][0]['links'][-1]['length']
)

full_ana_time = 0
print(f"{len(e['entries'])} episodes")

for entry in e['entries']:
    if 'itunes_duration' not in entry:
        print(f"missing itunes_duration: {entry['title']}")
        continue
    
    entry_duration = int(entry['itunes_duration'])
    sanity_check_duration(entry_duration, title=entry['title'])

    full_ana_time += entry_duration

full_ana_time

140 episodes


718156

In [36]:
f = feedparser.parse(SRC_RSS_SQSS)

(f['feed']['title'],
 f['entries'][0]['title'],
 f['entries'][0]['itunes_duration'],
 f['entries'][0]['links'][-1]['length']
)

full_sqss_time = 0
print(f"{len(f['entries'])} episodes")

for entry in f['entries']:
    if 'itunes_duration' not in entry:
        print(f"missing itunes_duration: {entry['title']}")
        continue

    entry_duration = int(entry['itunes_duration'])
    sanity_check_duration(entry_duration, title=entry['title'])

    full_sqss_time += entry_duration

full_sqss_time

37 episodes


294347

In [37]:
g = feedparser.parse(SRC_RSS_FOD)

(g['feed']['title'],
 g['entries'][0]['title'],
 g['entries'][0].get('itunes_duration'),
 g['entries'][0]['links'][-1]['length']
)

full_fod_time = 0
print(f"{len(g['entries'])} episodes")

for entry in g['entries']:
    if 'The Pathfinder Playtest' not in entry['title']:
        continue
    
    if 'itunes_duration' not in entry:
        print(f"missing itunes_duration: {entry['title']}")

        entry_duration = get_duration_for_entry(entry)    
        full_fod_time += entry_duration
        print(f"ffprobe found duration {entry_duration}")
        
        continue
    
    entry_duration = int(entry['itunes_duration'])
    sanity_check_duration(entry_duration, title=entry['title'])

    full_fod_time += entry_duration

full_fod_time

100 episodes
missing itunes_duration: The Pathfinder Playtest Part 7
ffprobe found duration 6055
missing itunes_duration: The Pathfinder Playtest Part 6
ffprobe found duration 3231
missing itunes_duration: The Pathfinder Playtest Part 5
ffprobe found duration 4412
missing itunes_duration: The Pathfinder Playtest Part 4
ffprobe found duration 3866
missing itunes_duration: The Pathfinder Playtest Part 3
ffprobe found duration 4140
missing itunes_duration: The Pathfinder Playtest Part 2
ffprobe found duration 5188
missing itunes_duration: The Pathfinder Playtest Part 1
ffprobe found duration 5008


31900

----
# Patreon RSS Feed

In [47]:
SRC_RSS_PATREON = os.environ.get("PATREON_RSS_FEED", "UNDEFINED")

ALL_PATREON_TITLES = [
    'Raiders of the Lost Continent',
    'Dinner at Lionlodge',
    'Blood & Blades',
    'Legacy of the Ancients',
    'Echo Quest',
    'New Game, Who Dis? Episode 1',
    'New Game, Who Dis? Episode 2',
    'New Game, Who Dis? Episode 3',
    'New Game, Who Dis? Episode 4',
    'New Game, Who Dis? Episode 5',
    'New Game, Who Dis? Episode 6',
    'New Game, Who Dis? Episode 7',
    'New Game, Who Dis? Episode 8',
    'New Game, Who Dis? Episode 9',
    'New Game, Who Dis? Episode 10',
    'New Game, Who Dis? Episode 11',
    'New Game, Who Dis? Episode 12',
    'New Game, Who Dis? Episode 13',
    'New Game, Who Dis? Episode 14',
    'New Game, Who Dis? Episode 15',
    'New Game, Who Dis? Episode 16',
    'New Game, Who Dis? Episode 17',
    'New Game, Who Dis? Episode 18',
    'New Game, Who Dis? Episode 19',
    'New Game, Who Dis? Episode 20',
    'New Game, Who Dis? Episode 21',
    'New Game, Who Dis? Episode 22',
    'Disorganized Play',
]

In [50]:
h = feedparser.parse(SRC_RSS_PATREON)

(h['feed']['title'],
 h['entries'][0]['title'],
 h['entries'][0].get('itunes_duration'),
 h['entries'][0]['links'][-1]['length']
)

full_patreon_time = 0
print(f"{len(h['entries'])} episodes")

for entry in h['entries']:
    if all([show_title not in entry['title'] for show_title in ALL_PATREON_TITLES]):
        print(f"skipping non-show episode {entry['title']}")
        continue
    
    if 'itunes_duration' not in entry:
        print(f"missing itunes_duration: {entry['title']}")

        entry_duration = get_duration_for_entry(entry)    
        full_patreon_time += entry_duration
        print(f"ffprobe found duration {entry_duration}")
        
        continue
    
    entry_duration = int(entry['itunes_duration'])
    sanity_check_duration(entry_duration, title=entry['title'])

    full_patreon_time += entry_duration

full_patreon_time

322 episodes
skipping non-show episode Cannon Fodder Friday 8/13/2021
missing itunes_duration: Raiders of the Lost Continent Season 3 Episode 14 - The Color of Honey
ffprobe found duration 3868
skipping non-show episode New Game, Who Dis? Dungeon Crawl Classics Episode 1
missing itunes_duration: Raiders of the Lost Continent Season 3 Episode 13 - Making a Bee Line
ffprobe found duration 3588
skipping non-show episode Cannon Fodder Friday 8/5/21
skipping non-show episode Glass Cannon Live! Washington D.C. 2021 - Strange Aeons Session 22
skipping non-show episode New Game, Who Dis? Marvel Super Heroes Episode 3
skipping non-show episode Cannon Fodder Friday 7/30/21
missing itunes_duration: Raiders of the Lost Continent Season 3 Episode 12 - Light Entertainment
ffprobe found duration 3830
skipping non-show episode New Game, Who Dis? Marvel Super Heroes Episode 2
skipping non-show episode Cannon Fodder Friday 7/23/21
missing itunes_duration: Raiders of the Lost Continent Season 3 Episode 1

ffprobe found duration 8119
skipping non-show episode New Game, Who Dis? Call of Cthulhu Episode 1
skipping non-show episode Cannon Fodder Friday 2/19/21
missing itunes_duration: Raiders of the Lost Continent Season 2 Episode 37 - Recant Always Get What You Want
ffprobe found duration 5058
missing itunes_duration: Blood & Blades: The Tin Whistles Episode 4
ffprobe found duration 8631
skipping non-show episode New Game, Who Dis? Tales from the Loop Episode 3
skipping non-show episode Cannon Fodder Friday 2/12/21
missing itunes_duration: Legacy of the Ancients Season 1 Episode 41 - Crustacean Identification
ffprobe found duration 4578
missing itunes_duration: Blood & Blades: The Tin Whistles Episode 3
ffprobe found duration 8176
missing itunes_duration: Blood & Blades: The Tin Whistles Episode 2
ffprobe found duration 7337
missing itunes_duration: Blood & Blades: The Tin Whistles Episode 1
ffprobe found duration 8986
skipping non-show episode New Game, Who Dis? Tales from the Loop Episod

ffprobe found duration 6804
missing itunes_duration: Legacy of the Ancients Season 1 Episode 4 - Hero Dork Nerdy
ffprobe found duration 3894
missing itunes_duration: New Game, Who Dis? Episode 7 - Stars Wars Age of Rebellion RPG Part 3
ffprobe found duration 4215
missing itunes_duration: Legacy of the Ancients Season 1 Episode 3 - The Aristocrat
ffprobe found duration 3879
missing itunes_duration: Legacy of the Ancients Season 1 Episode 2 - Fire Festival
ffprobe found duration 3842
missing itunes_duration: Legacy of the Ancients Season 1 Episode 1 - Morning in Sandpoint
ffprobe found duration 3827
missing itunes_duration: New Game, Who Dis? Episode 6 - Stars Wars Age of Rebellion RPG Part 2
ffprobe found duration 4146
missing itunes_duration: New Game, Who Dis? Episode 5 - Star Wars Age of Rebellion
ffprobe found duration 8587
missing itunes_duration: New Game, Who Dis? Episode 4 - Paranoia Part 2
ffprobe found duration 9228
missing itunes_duration: New Game, Who Dis? Episode 3 - Paran

ffprobe found duration 4058
missing itunes_duration: Raiders of the Lost Continent Season 1 Episode 32 - Elemental Fury
ffprobe found duration 4182
missing itunes_duration: Raiders of the Lost Continent Season 1 Episode 31 - Rise Up, Eliza
ffprobe found duration 4202
skipping non-show episode Glass Cannon Live! Philadelphia - Strange Aeons Session #2
skipping non-show episode Glass Cannon Live! LA - Strange Aeons Session #1
missing itunes_duration: Raiders of the Lost Continent Season 1 Episode 30 - Null and Droid
ffprobe found duration 4598
missing itunes_duration: Raiders of the Lost Continent Season 1 Episode 29 - Robots of Brawn
ffprobe found duration 3842
missing itunes_duration: Raiders of the Lost Continent Season 1 Episode 28 - Tower Corrupts
ffprobe found duration 4004
missing itunes_duration: Raiders of the Lost Continent Season 1 Episode 27 - Marble Madness
ffprobe found duration 4662
missing itunes_duration: Raiders of the Lost Continent Season 1 Episode 26 - Class Warfare


911861

----
# Result

In [56]:
public_audio_content = full_gcp_time + full_ana_time + full_sqss_time + full_fod_time
public_video_content = full_video_time

print('Public Content Duration')
timedelta_public = timedelta(seconds=(public_audio_content + public_video_content))
timedelta_public

Public Content Duration


datetime.timedelta(days=35, seconds=37112)

In [57]:
print('Patreon Content Duration')
timedelta_patreon = timedelta(seconds=full_patreon_time)
timedelta_patreon

Patreon Content Duration


datetime.timedelta(days=10, seconds=47861)

In [58]:
timedelta_total = timedelta_public + timedelta_patreon
timedelta_total.days * 24 + timedelta_total.seconds / 60 / 60

1103.6036111111111